# Data Acquisition

Calling the Genius API directly(not using)

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import re

import pandas as pd
import numpy as np

In [2]:
GENIUS_API_TOKEN="2AaY5B5Iylvf3X2oQl_4Jmk74pSZ-J8VqcboMk1PO_y7N8wlN83D9a85uhNh49Id"

In [4]:
song_info = []
songs = []
pulled_lyrics = []

In [5]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response
# Get Genius.com song url's from artist object
def request_song_url(artist_name, song_cap):
    page = 1

    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        # Collect up to song_cap song objects from artist

        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs


In [6]:
# Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()
    #remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    #remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])         
    return lyrics

# DEMO
# print(scrape_song_lyrics('https://genius.com/Jason-derulo-talk-dirty-lyrics'))

In [7]:
def write_lyrics_to_file(artist_name, song_count):
#     f = open('lyrics/' + artist_name.lower() + '.txt', 'wb')
    urls = request_song_url(artist_name, song_count)
    for url in urls:
        lyrics = scrape_song_lyrics(url)
#         f.write(lyrics.encode("utf8"))
        pulled_lyrics.append(lyrics)
#     f.close()
#     num_lines = sum(1 for line in open('lyrics/' + artist_name.lower() + '.txt', 'rb'))
#     print('Wrote {} lines to file from {} songs'.format(num_lines, song_count))
  
# DEMO  


In [8]:
artist = []
feature = []
title = []
lyrics = []

In [9]:
artists = ['Jason Derulo', 'Cake', 'System of A Down']
#, 'Nicki Minaj', 'Britney Spears', 'BLACKPINK', 'Will.i.am', 'Lady Gaga', 'Shakira', 'Ludacris', 'Akon', 'Sean Paul', 'Usher', 'Dua Lipa', 'The Weeknd', 'Cardi B', 'Miley Cyrus', 'Logic', 'Post Malone', 'Ini Kamoze', 'Rachel Platten', 'Sia', 'Muse', 'Maroon 5', 'Christina Perri', 'Lorde', 'John Newman', 'Bruno Mars', 'P!nk', 'Lana Del Rey', 'Keane', 'Deadmau5', 'The Proclaimers', 'Mumford & Sons', 'Run The Jewels', 'Fischerspooner', 'Mos Def', 'Pharell Williams', 'Weird Al Yankovic', 'Tones and I', 'Billie Eilish', 'Ariana Grande', 'Megan Thee Stallion', 'Doja Cat', 'DaBaby', 'Halsey'

In [10]:
even_more = ['rick astley', '']

In [ ]:
for a in artists: 
    write_lyrics_to_file(a, 10)

In [ ]:
# write_lyrics_to_file('cake', 100)

In [ ]:
for song in songs:
    for info in song_info:
        if song in info['result']['url']:
            try:
                artist.append(info['result']['primary_artist']['name'])
                title.append(info['result']['title'])
                feature.append(re.sub(r'^[^\(]+', '', info['result']['title_with_featured'].replace('\xa0', ' ')))

            except:
                artist.append(np.NaN)
                title.append(np.NaN)
                feature.append(np.NaN)
                
# for lyric in pulled_lyrics:
#     try:
#         lyrics.append(lyric.replace('\n', ' ').replace('\r', '').replace("\'", ''))
#     except:
#         lyrics.append(np.NaN)


In [ ]:
len(artist)

In [ ]:
len(title)

In [ ]:
len(feature)

In [ ]:
len(lyrics)

In [ ]:
lyrics[149]

In [21]:
df = pd.DataFrame({
'Artist' : artist,
'Featuring' : feature,
'Title' : title})

In [30]:
df[df.duplicated(keep=False)].value_counts('title')

AttributeError: 'DataFrame' object has no attribute 'value_counts'